In [8]:
from call_nlp_api import *
from google.cloud import bigquery

import warnings
warnings.simplefilter("always", category=PendingDeprecationWarning) 
warnings.simplefilter("always", category=DeprecationWarning) 

In [9]:
"""Queries BQ, adds NL API results, and writes back to BQ."""
bq_client = bigquery.Client(location="US", project="ut-goog")
query = """
  SELECT 
    complaint_id, consumer_complaint_narrative
  FROM `ut-goog.nl_api.sentiment_data`;
"""
dataset = "nl_api"
table_name = "entity_data"
entities_path = "./entities"

# Load queries into dataframe and load complaint checkpoints.
df = load_query_results_to_df(bq_client, query)

Query results loaded into dataframe!


In [11]:
entities = load_checkpoint(entities_path)

entities = call_nl_api(df, entities, entities_path,get_entities, batch_size=2500)

# Add NL API data to dataframes and write to BQ.
df = add_entity_data(df, entities)

400 The language fr is not supported for entity_sentiment analysis.
[./entities] count: 2500
Processed 2500 rows in 29.206058025360107 seconds.
[./entities] count: 5000
Processed 2500 rows in 29.423500299453735 seconds.
[./entities] count: 7500
Processed 2500 rows in 27.75111746788025 seconds.
[./entities] count: 10000
Processed 2500 rows in 25.303362131118774 seconds.
[./entities] count: 12500
Processed 2500 rows in 22.654243230819702 seconds.
[./entities] count: 15000
Processed 2500 rows in 21.003156185150146 seconds.
[./entities] count: 17500
Processed 2500 rows in 23.97548484802246 seconds.
[./entities] count: 20000
Processed 2500 rows in 30.73464846611023 seconds.
[./entities] count: 22500
Processed 2500 rows in 19.177119970321655 seconds.
[./entities] count: 25000
Processed 2500 rows in 21.92923092842102 seconds.
[./entities] count: 27500
Processed 2500 rows in 31.68334650993347 seconds.
[./entities] count: 30000
Processed 2500 rows in 24.298340559005737 seconds.
[./entities] cou

In [14]:
len(df)

496191

In [15]:
df.to_csv("entity_data.csv")

In [16]:
df.index = df.index.astype(int)
dataset = bq_client.dataset(dataset)
table_ref = dataset.table(table_name)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: PendingDeprecationWarning: Client.dataset is deprecated and will be removed in a future version. Use a string like 'my_project.my_dataset' or a cloud.google.bigquery.DatasetReference object, instead.
  


In [17]:
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = "WRITE_TRUNCATE"
job = bq_client.load_table_from_dataframe(df, table_ref, location="US",
                                            job_config=job_config)
job.result()
assert job.state == "DONE"
print("BQ table updated:", table_name)

/home/jupyter/.local/lib/python3.5/site-packages/google/cloud/bigquery/_pandas_helpers.py:391: UserWarning: Pyarrow could not determine the type of columns: entities, entity_types, entity_sentiment_scores, entity_sentiment_magnitudes.
  ", ".join(field.name for field in unknown_type_fields)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: PendingDeprecationWarning: Schema could not be detected for all columns. Loading from a dataframe without a schema will be deprecated in the future, please provide a schema.
  after removing the cwd from sys.path.


BQ table updated: entity_data


In [18]:
df.head()

,consumer_complaint_narrative,entities,entity_types,entity_sentiment_scores,entity_sentiment_magnitudes
complaint_id,,,,,
2824350,DBA of transaction says dr due to ATM/DEP erro...,"[transaction, DBA, dr, error, XX/XX, DEP, ATM,...","[OTHER, OTHER, PERSON, OTHER, OTHER, ORGANIZAT...","[0, 0, 0, -0.699999988079071, -0.1000000014901...","[0, 0, 0, 0.699999988079071, 0.100000001490116..."
2051787,They cal upwards of 15 times a day.,[times],[OTHER],[0],[0]
3418791,Alot of hard impacts are effecting my credit.,"[Alot, impacts, credit]","[OTHER, OTHER, OTHER]","[-0.5, -0.6000000238418579, 0]","[0.5, 0.6000000238418579, 0]"
2560181,XXXX {$510.00} DISPUTED LETTERS SEVERAL TIMES ...,"[LETTERS, XXXX, TIMES XXXX, TIMES]","[WORK_OF_ART, ORGANIZATION, PERSON, OTHER]","[-0.20000000298023224, -0.30000001192092896, -...","[0.5, 0.30000001192092896, 0.10000000149011612..."
2684583,My interest has compounded to the point I can ...,"[interest, point, loans]","[OTHER, OTHER, OTHER]","[-0.5, -0.10000000149011612, 0]","[0.5, 0.10000000149011612, 0]"
